# Personalized Programming Tutor - Week 1 Project

## Project Objective

Build a **personalized tutor** that takes a technical question about code and responds with a detailed and educational explanation.

### What does this tutor do?

- Explains complex code in a simple and structured way
- Provides examples and analogies
- Teaches the "why" behind code patterns
- Responds in well-formatted Markdown
- Supports **streaming** to see responses in real-time

### Features:

- **Flexible**: Switch between Ollama (local, free) and OpenAI (cloud)
- **Interactive**: Streaming mode with typewriter effect
- **Educational**: Designed specifically for learning

---

**Instructions**: Execute the cells in order and modify the `question` variable to ask new questions.

In [1]:
# Imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [2]:
# CONFIGURATION: Change here to test different providers
# Options: 'ollama' or 'openai'
USE_PROVIDER = 'ollama'  # Change to 'openai' to use OpenAI

print(f"Selected provider: {USE_PROVIDER.upper()}")

Selected provider: OLLAMA


In [23]:
# Initialize and constants

# Load configuration from global .env
load_dotenv(dotenv_path='/workspace/.env', override=True)

# Use provider selected in previous cell (or from .env as fallback)
llm_provider = USE_PROVIDER if 'USE_PROVIDER' in globals() else os.getenv('LLM_PROVIDER', 'ollama')

if llm_provider == 'ollama':
    # OLLAMA CONFIGURATION (Local)
    ollama_base_url = os.getenv('OLLAMA_BASE_URL')
    ollama_api_key = os.getenv('OLLAMA_API_KEY')
    #ollama_model = os.getenv('OLLAMA_MODEL')
    ollama_model = 'qwen3-coder:480b-cloud'
    
    print("=" * 50)
    print("Using OLLAMA (Local)")
    print("=" * 50)
    print(f"   Base URL: {ollama_base_url}")
    print(f"   Model: {ollama_model}")
    print(f"   Cost: FREE")
    print("=" * 50)
    
    # Create OpenAI client pointing to Ollama
    openai = OpenAI(
        base_url=f"{ollama_base_url}/v1",
        api_key=ollama_api_key
    )
    MODEL = ollama_model
    
else:
    # OPENAI CONFIGURATION (Cloud)
    api_key = os.getenv('OPENAI_API_KEY')
    
    print("=" * 50)
    print("Using OPENAI (Cloud API)")
    print("=" * 50)
    
    if api_key and api_key.startswith('sk-proj-') and len(api_key) > 50:
        print(f"   API Key: sk-proj-...{api_key[-8:]}")
        print(f"   Model: gpt-4o-mini")
        print(f"   Cost: ~$0.15 / 1M tokens input")
        print("   Status: Configured correctly")
    else:
        print("   Status: Invalid or missing API Key")
        print("   Check the .env file")
    
    print("=" * 50)
    
    openai = OpenAI(api_key=api_key)
    MODEL = 'gpt-4o-mini'

Using OLLAMA (Local)
   Base URL: http://192.168.80.200:11434
   Model: qwen3-coder:480b-cloud
   Cost: FREE


In [18]:
# Call LLM API with streaming support

def callModel(ask, use_stream=False):
    """
    Calls the LLM model with the provided messages
    
    Args:
        ask: List of messages with format [{"role": "...", "content": "..."}]
        use_stream: If True, displays response in real-time (typewriter effect)
    
    Returns:
        The complete response content
    """
    if use_stream:
        stream = openai.chat.completions.create(
            model=MODEL,
            messages=ask,
            stream=True  # Enable streaming
        )
        
        response = ""
        display_handle = None
        
        for chunk in stream:
            content = chunk.choices[0].delta.content or ''
            response += content
            
            if display_handle is None and response:
                display_handle = display(Markdown(response), display_id=True)
            elif display_handle is not None:
                update_display(Markdown(response), display_id=display_handle.display_id)
        
    else:
        response = openai.chat.completions.create(
            model=MODEL,
            messages=ask
        )
        result = response.choices[0].message.content
        display(Markdown(result))

In [19]:
# Define our system prompt - PERSONALIZED TUTOR

system_prompt = """
You are an expert programming tutor with deep knowledge in Python, JavaScript, and software engineering.

Your teaching style:
- Explain concepts clearly and step by step
- Use analogies and real-world examples
- Break down complex code into understandable parts
- Highlight best practices and common pitfalls
- Encourage learning by explaining the "why" behind the code

Response format:
- Always respond in well-formatted Markdown
- Use headers, lists, and code blocks appropriately
- Do NOT wrap your entire response in markdown code fences (```)
- Include examples when helpful
- Be concise but thorough

Your goal is to help students truly understand programming concepts, not just memorize syntax.
"""

In [20]:
# Get the technical question from user input

print("=" * 60)
print("PERSONALIZED PROGRAMMING TUTOR")
print("=" * 60)
print("\nEnter the code you want to understand.")
print("\nExample:")
print('  yield from {book.get("author") for book in books if book.get("author")}')
print("\nYour question (paste your code below):")
print("-" * 60)

question = input()

PERSONALIZED PROGRAMMING TUTOR

Enter the code you want to understand.

Example:
  yield from {book.get("author") for book in books if book.get("author")}

Your question (paste your code below):
------------------------------------------------------------


 for chunk in stream:             content = chunk.choices[0].delta.content or ''             response += content


In [21]:
# Define user prompt using the question variable

user_prompt = f"""
Please explain the following code in detail:

```python
{question}
```

Include:
1. What this code does
2. How it works step by step
3. Why someone would use this approach
4. Any important concepts or patterns involved
"""

# Build messages for the LLM
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

In [24]:
# Execute the tutor with the question

# Option 1: Without streaming (shows complete result at the end)
# callModel(messages)

# Option 2: With streaming (typewriter effect - recommended)
callModel(messages, use_stream=True)

# Streaming Response Handling Code Explanation

## 1. What This Code Does

This code processes a **streaming response** from an AI language model (like ChatGPT). It extracts text content from each chunk of the streamed response and builds up a complete response string incrementally.

**Note**: The code appears to be missing proper line breaks. Here's the corrected version:

```python
for chunk in stream:
    content = chunk.choices[0].delta.content or ''
    response += content
```

## 2. Step-by-Step Breakdown

### Step 1: `for chunk in stream:`
- **What**: Iterates through each data chunk in a streaming response
- **Analogy**: Like watching a video that loads progressively - you get small pieces of data as they become available
- **Why streaming**: Instead of waiting for the complete response, you get it piece by piece

### Step 2: `chunk.choices[0].delta.content or ''`
- **`chunk.choices[0]`**: Accesses the first (usually only) choice/response option
- **`.delta`**: Contains the incremental change/addition to the response
- **`.content`**: The actual text content of this incremental update
- **`or ''`**: Handles cases where `content` might be `None` (returns empty string instead)

### Step 3: `response += content`
- **What**: Appends the new content to the existing response string
- **Result**: Builds the complete response progressively, chunk by chunk

## 3. Why Use This Approach

### Real-time User Experience
```
Instead of: [Wait 10 seconds] → "Hello, how can I help you today?"
Streaming:   "Hello," → ", how" → " can I" → " help" → " you today?"
```

### Memory Efficiency
- Processes data as it arrives
- Doesn't need to store the entire response in memory at once
- Enables handling very long responses

### Better User Engagement
- Users see progress immediately
- Reduces perceived waiting time
- Enables interactive applications

## 4. Important Concepts and Patterns

### Streaming Pattern
```python
# General streaming pattern
for data_chunk in data_source:
    process(chunk)
    accumulate_results()
```

### Safe Property Access
```python
# The `or ''` pattern prevents None-related errors
content = potentially_none_value or ''  # Python's truthiness at work
```

### Incremental Processing
- Common in APIs, file reading, and real-time data processing
- Enables responsive applications
- Reduces latency perception

### Typical Usage Context
```python
# Complete example context
response = ""  # Initialize empty response
stream = openai.ChatCompletion.create(..., stream=True)

for chunk in stream:
    content = chunk.choices[0].delta.content or ''
    response += content
    print(content, end='', flush=True)  # Print in real-time
```

This approach is essential for creating responsive AI applications and demonstrates key concepts in asynchronous programming and real-time data handling.

------------------------------------------------------------------------------------------------------------------------------

## How to Use This Personalized Tutor

### Quick Start Guide:

1. **Setup (one time)**
   - Execute cells 2-6 to configure the LLM provider and initialize the tutor

2. **Ask your question**
   - Execute cell 7
   - When prompted, paste or type the code you want to understand
   - Press Enter

3. **Get the explanation**
   - Execute cell 8 to build the prompt
   - Execute cell 9 to see the AI tutor's explanation
   - Use `use_stream=True` for real-time typewriter effect

### Tips:

- You can ask about any programming concept or code snippet
- The tutor works best with Python, JavaScript, and general programming patterns
- For new questions, simply re-execute cells 7, 8, and 9
- Switch between Ollama (free, local) and OpenAI (cloud) by changing cell 3

### Example Questions:

```python
# Generators and yield
yield from {book.get("author") for book in books if book.get("author")}

# List comprehensions
[x**2 for x in range(10) if x % 2 == 0]

# Decorators
@decorator
def function(): pass

# Context managers
with open("file.txt") as f:
    data = f.read()

# Lambda and functional programming
list(map(lambda x: x * 2, [1, 2, 3]))

# Async/await
async def fetch_data():
    await asyncio.sleep(1)
```